<div dir="rtl" align="center">
    <h1>
        تمرین اول درس پردازش زبان‌های طبیعی
    </h1>
    <h3>
        گردآورندگان:<br/>
        ساحل مس‌فروش، سروش تابش، درنا دهقانی
    </h3>
    
</div>

<br/>
<br/>


<div dir="rtl">
    ما در این تمرین ترک "استخراج پرسش و پاسخ" را انتخاب کردیم. در این تمرین قصد داریم از یک جمله فارسی سوالاتی با توجه به روابط علت و معلولی، نقش کلمات در جملات، ارتباط بین واژگان و ... استخراج کنیم. هر سوال به شکل یک دیکشنری پایتون نمایش داده می‌شود که key سوال و value پاسخ به سوال است.
</div>

<div dir="rtl">
    <h2> 1. تمیز کردن ورودی </h2>
    در این بخش، گام‌های ابتدایی برای مراحل آتی را برمی‌داریم. هر جمله که به عنوان ورودی ما داده می‌شود، باید تا حد مناسبی تمیز و ساختاریافته شود تا به عنوان ورودی بخش‌های بعد معتبر باشد. کارهایی مانند نرمال‌سازی و ... در این بخش انجام می‌گردد. از توابع تعریف شده در کتابخانه hazm استفاده می‌کنیم.
</div>

In [41]:
import re
from collections import Counter
import hazm
import json
import os
from itertools import chain
import collections
from hazm import *

In [53]:
def sentence_clean(sentence: str):
    normalizer = Normalizer()
    return normalizer.normalize(sentence)

sentence = ''
print(sentence_clean(sentence))

<div dir="rtl">
    <h2> 2. استخراج ساختار سوالات </h2>
    جملاتی که امکان استخراج سوالات از آنها ممکن است، دو نوعند. نوع اول دارای برخی کلمات کلیدی (مانند زیرا، چون و ...) هستند. نوع دوم جملاتی هستند که فاقد اینگونه کلماتند. مثلاً امروز هوا بارانیست: امروز هوا چگونه است؟ بارانی.<br>
    <h4> 2.1. استخراج سوال از جملات با کلمات کلیدی </h4>
    در این مرحله، به یافتن کلمات کلیدی در جملاتی که امکان استخراج سوالات از آنها وجود دارد پرداختیم. در یک Google Sheet لیست این کلمات، انواع نحوه به کارگیری آنها در جملات، مثال‌هایی از هر یک و هم‌چنین ساختار سوال و جواب و مثال از آنها را مشخص کردیم. 
    هم‌چنین مثال‌هایی از جملات را به عنوان داده‌ی تست یافتیم و سوال‌هایی از آنها استخراج کردیم.
    <a href="https://docs.google.com/spreadsheets/d/1qsCfRVr5RgXYW2qEVbrP3dq7-l3wNngjPAsXrHN89DQ/edit#gid=0" target="_top">فهرست کلمات کلیدی و جملات تست</a><br/>
    سپس در تابع <code>extract_cause_effect</code> از جمله‌ای که به عنوان ورودی داده می‌شود، با توجه به ساختار جمله که ورودی دیگر این تابع و از انواع پرسش و پاسخ است، رابطه‌ی علت و معلولی که ممکن است در جمله باشد، استخراج می‌گردد.<br>
    در دیکشنری <code>all_regexes</code> نوع و ساختار جملاتی که قابلیت طرح سوال از آنها وجود دارد توسط regex طراحی شده است. هم‌چنین ساختار پرسش و پاسخ آنها نیز وجود دارد.<br>
    اگر ساختار regex در جمله موجود بود، بسته به اینکه کدام بخش جمله در سوال و کدام بخش آن در جواب استفاده شود، سوال و جواب از جمله ساخته شده و در دیکشنری خروجی تابع قرار می‌گیرد.
</div>

In [39]:
all_regexes = {
  "چرایی": {"regex": r"""(?P<effect>.*)[،|؛] (?P<reason> زیرا|چون|به‌دلیل|بدلیل|چون\sکه|به\sدلیل\sاینکه) (?P<cause>.*)|چون(?P<cause2>.*)(،|\s،|؛|\s؛)(?P<effect2>.*)""", "question": "چرا {effect} ؟", "answer": "زیرا {cause}"}, 
  "سببی": {"regex": r"(?P<effect>.*) (?P<reason> دلیل|مسبب) (?P<cause>.*) است.|(?P<reason2>مسبب|دلیل) (?P<effect2>.*)[،|؛] (?P<cause2>) است.", "question": "{effect} باعث چه می شود؟ ", "answer": "باعث {cause}"},
  "موجب": {"regex":r"(?P<effect>.*) موجب (?P<cause>.*) می‌شود.*|(?P<effect2>.*) موجبات (?P<cause2>.*) فراهم می‌کند", "question": "چه موجب {effect} می‌شود؟", "answer": "{cause}"},
  "ترین": {"regex":r"(?P<tarin>.*ترین) (?P<cause>.*)[،|؛](?P<effect>.*) است", "question": "{tarin}ترین {cause} چیست؟", "answer": "{effect}"},
  "خاطر": {"regex": r"""(?P<reason> بخاطر|به خاطر) (?P<cause>.*)[،|] (?P<effect>.*)|(?P<effect2>.*) (?P<reason2> بخاطر|به خاطر) (?P<cause2>.*)""", "question": "چرا {effect} ؟", "answer": "به خاطر {cause}"},
  "منجر": {"regex": r"(?P<cause>.*)(?P<reason> منجر به)(?P<effect>.*)می شود.|(?P<reason2>مسبب|دلیل) (?P<effect2>.*)[،|؛] (?P<cause2>) است.", "question": "علت {effect} چیست؟", "answer": "{cause}"},
   "هدف": {"regex":r"هدف (?P<effect>.*) (?P<reason>این\sاست\sکه|این\sاست) (?P<cause>.*)", "question": "هدف {effect} چیست؟", "answer": "{cause}"},
  #  "": {"regex":r"", "question": "", "answer": ""},
}

def extract_cause_effects(text: str, key: str):
    """
    extracting cause and effect parts from text based on key regex. creating a Q&A based on key question format and answer format
    Parameters
    ---------
    text: str
        sentence query used to extract question and answers.
    key: str
        key should be one of all_regexes.keys(). sentence format used to extract Q&A
  
  
    Returns
    -------
    result: dict
        includes question and answer sentences.
    """
    
    print("in func for: ", key)
    key_information = all_regexes.get(key, None)
    if not key_information:
        print("you should specify current key information in all_regexes dict")
        return
    regex = key_information.get("regex")
    first_reg = re.compile(regex)
    x = re.search(first_reg, text)
    if x is not None:
        cause = x.group("cause")
        effect = x.group("effect")
        if not cause:
        # text is not in the format of first part of regex. checking for second part
            cause = x.group("cause2")
            effect = x.group("effect2")
            if not cause or not effect:
                # cause and effects are empty! this shouldn't happen!!!
                print("bad input")
                return None
    else:
        # regex didn't found the structure. 
        return None
    q_format = key_information.get("question")
    a_format = key_information.get("answer")  
    result = {"question": sentence_clean(q_format.format(effect=effect)), "answer": sentence_clean(a_format.format(cause=cause))}
    return result

<div dir="rtl">
    در تابع <code>extract_question</code> ابتدا روی تمامی ساختارهای موجود که امکان طرح سوال از آنها وجود دارد، پیمایشی انجام می‌گیرد. در هر پیمایش، هر ساختار از <code>all_regexes</code> در جمله‌ی ورودی توسط تابع <code>extract_cause_effects</code> بررسی می‌گردد و در صورت داشتن خروجی معتبر، پرسش و پاسخ مربوطه‌ی آن نمایش داده می‌شود.
</div>

In [40]:
def extract_questions(text: str):
    """
    checking all different sentence formats on text. if one is applied ignore others and return
    Parameters
    ----------
    text: str
        input sentence
    
    Returns
    ------
    result: dict
      question and answer sentences.
    """
    for key, value in all_regexes.items():
        result = extract_cause_effects(text, key)
        if result:
            return result
    
sentence = 'رنگین کمونه؛ چون بارون اومده.'
print(extract_questions(sentence_clean(sentence)))

sentence = 'چون بارون اومده؛ رنگین کمونه.'
print(extract_questions(sentence_clean(sentence)))

{'question': 'چرا رنگین کمونه؟', 'answer': 'زیرا بارون اومده.'}
{'question': 'چرا رنگین کمونه.؟', 'answer': 'زیرا بارون اومده'}


<div dir='rtl'>
    <h4>2.2. استخراج سوال از جملات بدون کلمات کلیدی</h4>
    عملکرد کلی در این بخش بدین صورت است که امکان پرسش با حذف بخشی از جملع ممکن است و به جای بخش حذف شده، ضمیر پرسشی مربوطه جایگزین آن می‌گردد و سوال تشکیل می‌شود. در نتیجه بخش حذف شده از جمله، پاسخ به آن پرسش است.<br>
    مثلاً در جمله‌ی "او کتاب را برداشت."، با حذف "او" که اشاره به فرد دارد و جایگزین کردن "چه کسی" یک سوال ساخته می‌شود. هم‌چنین با حذف "کتاب" که اشاره به شیء دارد و جایگزین کردن "چه چیزی" می‌توان سوال دیگری ساخت.<br>
    چالشی که در این بخش با آن مواجه بودیم، نحوه برخورد با گروه‌های اسمی بود که با حرف اضافه شروع می‌شدند. مثلاً در جمله‌ی "او با آرامش راه رفت." تشخیص اینکه "با" حرف ربط یا حرف اضافه است برای ما مسئله بود. در چنین شرایطی تصمیم گرفتیم به کلمه‌ی پیش از این حرف نگاه کنیم و اگر فعل بود، آن را حرف ربط و در غیر اینصورت حرف اضافه در نظر بگیریم. در این صورت کلمه‌ی پرسشی مناسب برای گروه اسمی "با آرامش"، "چگونه" است.
</div>

<div dir="rtl">
    <h2> 3. صحت‌سنجی </h2>
    ابتدا در فایل <code>question_generator.json</code> تعدادی مثال و خروجی‌های مورد توقع از آنها برای پرسش و پاسخ نوشته شد. سپس توسط کلاس <code>BaseTest</code> توابعی برای صحت‌سنجی عملکرد بخش‌های پیشین در یافتن و استخراج پرسش و پاسخ از جملات نوشته شد.
</div>

In [51]:
class BaseTest:

    def get_testcases(self, addr):
        f = open(os.getcwd()+addr, 'r')
        test_cases = json.load(f)
        f.close()
        return test_cases

    def run_test(self, obj, addr):
        errors = []
        test_cases = self.get_testcases(addr)
        for i in test_cases:
            your_answer = obj.run(i['input'])
            correct_answer = i['outputs']
            d = {k: (your_answer[k], correct_answer[k]) for k in your_answer if k in correct_answer and your_answer[k] != correct_answer[k]}
            for j in d:
                errors.append('Input {0}: your answer is {1} correct answer is {2}'.format(i['id'], d[j][0], d[j][1]))
        assert not errors, 'errors occured:\n{}'.format('\n'.join(errors))

    def test_json(self):
        test_cases = self.get_testcases('\question_generator.json')
        for i in test_cases:
            your_answer = extract_questions(i.get('input'))
            correct_answer = i.get('output')
            print("Testing {0}: ".format(str(i['id']) ))
            print("correct answer: ", correct_answer)
            print("method answer: ", your_answer)
            print("***********************************")

tester = BaseTest()
tester.test_json()

Testing 1: 
correct answer:  {'question': 'ع†ط±ط§ ط´ظ…ط§ ط¨ط§ ط°ع©ط±ظ‡ط§غŒ ط²غŒط± ط¨ظ‡ ط®ظˆط¨غŒ ط¢ط´ظ†ط§غŒغŒط¯?', 'answer': 'ط²غŒط±ط§ ط¯ط± ظ‡ط± ظ†ظ…ط§ط² ط¢ظ†ظ‡ط§ ط±ط§ طھع©ط±ط§ط± ظ…غŒع©ظ†غŒط¯'}
method answer:  None
***********************************
Testing 2: 
correct answer:  {'question': 'ع†ط±ط§ ط±ظ†ع¯غŒظ† ع©ظ…ظˆظ†ظ‡طں', 'answer': 'ع†ظˆظ† ط¨ط§ط±ظˆظ† ط§ظˆظ…ط¯ظ‡'}
method answer:  None
***********************************
Testing 3: 
correct answer:  {'question': 'ظ…ط³ط¨ظ‘ط¨ طھطµط§ط¯ظپ ع†غŒط³طھطں', 'answer': 'ط¨غŒ ط§ط\xadطھغŒط§ط·غŒ'}
method answer:  None
***********************************
Testing 4: 
correct answer:  {'question': 'ط¯ظ„غŒظ„ ظ…ظˆظپظ‚غŒطھ ط§ظˆ ع†غŒط³طھطں', 'answer': 'ظ¾ط´طھع©ط§ط± ط²غŒط§ط¯'}
method answer:  None
***********************************
Testing 5: 
correct answer:  {'question': 'ظ‡ط¯ظپ ظ…ط§ ط¯ط± ظ†ظ‡غŒ ط§ط² ظ…ظ†ع©ط± ع†غŒط³طھطں', 'answer': ' ع©ظ‡ ط§ظ†ط³ط§ظ† ط®ط·ط§ع©ط§ط± ط¨ظ‡ ط§ط´طھط¨ط§ظ‡ط§طھط´ ط§ط¯ط§ظ…ظ‡ ظ†ط¯ظ‡ط¯'}
method answer:  None
***************************